# Capstone Project - The Battle of the Neighborhoods (Week 2)
## Applied Data Science Capstone by IBM/Coursera

## [Introduction](#intro)
## [Data](#data)
## [Methodology](#methodology)
## [Results](#results)
## [Discussion](#discussion)
## [Analysis](#analysis)
## [Conclusion](#conclusion)

## Introduction: Business Problem <a id='intro' name="introduction"></a>

Hue and Da Nag are two cities in central Vietnam. They are both on the coast of Vietnam, but Da Nang is more renowned for beaches than Hue. 
Hue on the other hand is known for being the former center of power in Vietnam and was the seat of Royalty until 1947.  
Both a major tourist attractions, or at least were until Covid19. 
I will compare the two cities as they stand today.
In particular I will look at the tips and see if there is a correlation between the length of the tip and the rating given.


# Data<a id='data' name="data"></a>

In [68]:
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder

In [69]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import IFrame
print('Libraries imported.')

Libraries imported.


Find the latitude and longitude of Da Nang, Vietnam

In [70]:
address = 'Da Nang, VN'

geolocator = Nominatim(user_agent="da_nang_explorer")
location_danang = geolocator.geocode(address)
latitude_danang = location_danang.latitude
longitude_danang = location_danang.longitude
print('The geograpical coordinate of Da Nang is {}, {}.'.format(latitude_danang, longitude_danang))

The geograpical coordinate of Da Nang is 16.068, 108.212.


Find the latitude and longitude of Hue, Vietnam

In [71]:
address = 'Hue, VN'

geolocator2 = Nominatim(user_agent="hue_explorer")
location = geolocator2.geocode(address)
latitude_hue = location.latitude
longitude_hue = location.longitude
print('The geograpical coordinate of Hue is {}, {}.'.format(latitude_hue, longitude_hue))

The geograpical coordinate of Hue is 16.4638013, 107.5821911.


Lets look at the locations of Hue and Da Nang on a map

In [72]:
# create map of Da Nang and Hue using latitude and longitude values
map_cities = folium.Map(location=[((latitude_danang + latitude_hue)/2), ((longitude_danang + longitude_hue)/2)], zoom_start=10)


folium.Marker(
    location=[latitude_danang, longitude_danang],
    popup="Da Nang",
    icon=folium.Icon(icon="green"),
).add_to(map_cities)

folium.Marker(
    location=[latitude_hue, longitude_hue],
    popup="Hue",
    icon=folium.Icon(icon="green"),
).add_to(map_cities)
   
map_cities

FourSquare details

In [73]:
# The code was removed by Watson Studio for sharing.

Get some information from FourSquare about Da Nang

In [74]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_danang, 
    longitude_danang, 
    radius, 
    LIMIT)

In [75]:
results_danang = requests.get(url).json()
results_danang

{'meta': {'code': 200, 'requestId': '602b473cf3484c1e42562da5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Da Nang',
  'headerFullLocation': 'Da Nang',
  'headerLocationGranularity': 'city',
  'totalResults': 128,
  'suggestedBounds': {'ne': {'lat': 16.104000036000038,
    'lng': 108.249393644179},
   'sw': {'lat': 16.031999963999965, 'lng': 108.17460635582101}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '582b1746641e5c276f013971',
       'name': "Pizza 4P's",
       'contact': {},
       'location': {'address': '8 Hoàng Văn Thụ',
        'lat': 16.062674975021636,
        'lng': 108.2228449147913,
        'labeledLatLngs': [{'label': 'display',
          'lat': 16.06267

In [79]:
items_danang = results_danang['response']['groups'][0]['items']
items_danang[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '582b1746641e5c276f013971',
  'name': "Pizza 4P's",
  'contact': {},
  'location': {'address': '8 Hoàng Văn Thụ',
   'lat': 16.062674975021636,
   'lng': 108.2228449147913,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.062674975021636,
     'lng': 108.2228449147913}],
   'distance': 1302,
   'cc': 'VN',
   'city': 'Đà Nẵng',
   'state': 'Thành Phố Đà Nẵng',
   'country': 'Việt Nam',
   'formattedAddress': ['8 Hoàng Văn Thụ',
    'Đà Nẵng',
    'Thành Phố Đà Nẵng',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d1ca941735',
    'name': 'Pizza Place',
    'pluralName': 'Pizza Places',
    'shortName': 'Pizza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'check

In [80]:
#fetching omly relevant data from categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#flatten JSON, Normalize JSON to Dataframe
dataframe = json_normalize(items_danang) 

# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby_danang = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby_danang['venue.categories'] = nearby_danang.apply(get_category_type, axis=1)

# clean columns
nearby_danang.columns = [col.split('.')[-1] for col in nearby_danang.columns]

#replce NaN values with Not found in address
nearby_danang['address'] = nearby_danang['address'].fillna("Not found")
nearby_danang.head()   

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Pizza 4P's,Pizza Place,8 Hoàng Văn Thụ,16.062675,108.222845,"[{'label': 'display', 'lat': 16.06267497502163...",1302,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[8 Hoàng Văn Thụ, Đà Nẵng, Thành Phố Đà Nẵng, ...",NaN,NaN,NaN,582b1746641e5c276f013971
1,cộng cà phê,Coffee Shop,39,16.068063,108.223512,"[{'label': 'display', 'lat': 16.06806287945548...",1231,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[39 (Nguyễn Thái Học), Đà Nẵng, Thành Phố Đà N...",Nguyễn Thái Học,551105,NaN,5a26a41a31ac6c676705e94c
2,Sanouva Danang Hotel (Khách Sạn Sanouva Đà Nẵng),Hotel,"68 Phan Chau Trinh, Hai Chau Dist., Da Nang City",16.066620,108.220031,"[{'label': 'display', 'lat': 16.06661966448064...",872,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[68 Phan Chau Trinh, Hai Chau Dist., Da Nang C...",68 Phan Chau Trinh,550000,NaN,546b5fc2498e85d6109593cc
3,Nướng Đà Thành,BBQ Joint,Lô 1-2 B2.4 Nguyễn Văn Linh,16.060689,108.218386,"[{'label': 'display', 'lat': 16.06068929153090...",1062,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,[Lô 1-2 B2.4 Nguyễn Văn Linh (Huỳnh Thúc Kháng...,Huỳnh Thúc Kháng,NaN,NaN,5405d20d498eae2180a53eab
4,Thìa Gỗ Danang-style restaurant (Quán ăn ngon ...,Vietnamese Restaurant,37 Ba Đinh,16.076115,108.218690,"[{'label': 'display', 'lat': 16.07611473938317...",1152,VN,Đà Nẵng,Thành Phố Đà Nẵng,Việt Nam,"[37 Ba Đinh, Đà Nẵng, Thành Phố Đà Nẵng, Việt ...",NaN,551300,NaN,5aabf13ed552c7495c4b16cc


Lets draw a map with all the Da Nang venues we got

In [81]:
# creating map zoom at present Location
map_danang = folium.Map(location=[latitude_danang,longitude_danang],zoom_start=14)

# adding the current location to map
folium.CircleMarker(location=[latitude_danang,longitude_danang],radius=5,color='red',fill=True,fill_color='red').add_to(map_danang)
folium.Marker(location=[latitude_danang,longitude_danang],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map_danang)

# drawing radius of search
folium.Circle(location=[latitude_danang,longitude_danang],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map_danang)

# adding all the popular places around
for lat,lon,name,cate,addr,dist in zip(nearby_danang.lat,nearby_danang.lng,nearby_danang.name,nearby_danang.categories,nearby_danang.address,nearby_danang.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map_danang)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map_danang)

map_danang

Pull info on Hue venues

In [95]:

url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_hue, 
    longitude_hue, 
    radius, 
    LIMIT)

In [96]:
results_hue = requests.get(url2).json()
results_hue



{'meta': {'code': 200, 'requestId': '602b4aae4ff54e0befc5b6f4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Huế',
  'headerFullLocation': 'Huế',
  'headerLocationGranularity': 'city',
  'totalResults': 62,
  'suggestedBounds': {'ne': {'lat': 16.499801336000036,
    'lng': 107.619660190731},
   'sw': {'lat': 16.427801263999964, 'lng': 107.54472200926901}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd1ab4a5e0cce720acfa184',
       'name': 'La Residence Hue Hotel & Spa',
       'contact': {},
       'location': {'address': '5 Le Loi Street',
        'lat': 16.459127862940925,
        'lng': 107.58007233528453,
        'labeledLatLngs': [{'label': 'display',
          'lat'

In [97]:
items_hue = results_hue['response']['groups'][0]['items']
items_hue[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bd1ab4a5e0cce720acfa184',
  'name': 'La Residence Hue Hotel & Spa',
  'contact': {},
  'location': {'address': '5 Le Loi Street',
   'lat': 16.459127862940925,
   'lng': 107.58007233528453,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.459127862940925,
     'lng': 107.58007233528453}],
   'distance': 567,
   'postalCode': '530000',
   'cc': 'VN',
   'city': 'Huế',
   'state': 'Tỉnh Thừa Thiên-Huế',
   'country': 'Việt Nam',
   'formattedAddress': ['5 Le Loi Street',
    'Huế',
    'Tỉnh Thừa Thiên-Huế',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d132951735',
    'name': 'Hotel Pool',
    'pluralName': 'Hotel Pools',
    'shortName': 'Pool',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/pool_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats'

In [98]:
#fetching omly relevant data from categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#flatten JSON, Normalize JSON to Dataframe
dataframe = json_normalize(items_hue) 

# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby_hue = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby_hue['venue.categories'] = nearby_hue.apply(get_category_type, axis=1)

# clean columns
nearby_hue.columns = [col.split('.')[-1] for col in nearby_hue.columns]

#replce NaN values with Not found in address
nearby_hue['address'] = nearby_hue['address'].fillna("Not found")
nearby_hue.head()   

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,La Residence Hue Hotel & Spa,Hotel Pool,5 Le Loi Street,16.459128,107.580072,"[{'label': 'display', 'lat': 16.45912786294092...",567,530000,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[5 Le Loi Street, Huế, Tỉnh Thừa Thiên-Huế, Vi...",NaN,4bd1ab4a5e0cce720acfa184
1,Lạc Thiên,Vietnamese Restaurant,6A Đinh Tiên Hoàng,16.468668,107.585144,"[{'label': 'display', 'lat': 16.468668, 'lng':...",626,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[6A Đinh Tiên Hoàng (Trần Hưng Đạo), Huế, Tỉnh...",Trần Hưng Đạo,4d3ab780325ff04df0dc1e45
2,Nook Cafe & Bar,Breakfast Spot,7/34 Nguyen Tri Phuong,16.464730,107.593040,"[{'label': 'display', 'lat': 16.46473, 'lng': ...",1162,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[7/34 Nguyen Tri Phuong, Huế, Tỉnh Thừa Thiên-...",NaN,583848f313bb7732ce36366c
3,The Scarlett Boutique Hotel,Hotel,Not found,16.467528,107.594623,"[{'label': 'display', 'lat': 16.467528, 'lng':...",1390,NaN,VN,NaN,NaN,Việt Nam,[Việt Nam],NaN,57c5bd65498eb3403f51e7e6
4,Nina's Cafe,Vietnamese Restaurant,16/34 Nguyễn Trí Phương,16.464284,107.592762,"[{'label': 'display', 'lat': 16.46428415334502...",1129,NaN,VN,Huế,Tỉnh Thừa Thiên-Huế,Việt Nam,"[16/34 Nguyễn Trí Phương, Huế, Tỉnh Thừa Thiên...",NaN,4c5cdff56ebe2d7faa8bd12e


Lets draw a map of Hue with the venues on it

In [101]:
# creating map zoom at present Location
map_hue = folium.Map(location=[latitude_hue,longitude_hue],zoom_start=14)

# adding the current location to map
folium.CircleMarker(location=[latitude_hue,longitude_hue],radius=5,color='red',fill=True,fill_color='red').add_to(map_hue)
folium.Marker(location=[latitude_hue,longitude_hue],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map_hue)

# drawing radius of search
folium.Circle(location=[latitude_hue,longitude_hue],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map_hue)

# adding all the popular places around
for lat,lon,name,cate,addr,dist in zip(nearby_hue.lat,nearby_hue.lng,nearby_hue.name,nearby_hue.categories,nearby_hue.address,nearby_hue.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map_hue)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map_hue)

map_hue

Lets get the Hue venue id's and use them to grab the ratings and tips into a dataframe called hue_venue_ids

In [102]:
hue_venue_ids = nearby_hue.iloc[: , [14]].copy() 
hue_venue_ids = hue_venue_ids.reindex(hue_venue_ids.columns.tolist() + ['rating','tips'], axis=1)
hue_venue_ids

,id,rating,tips
0,4bd1ab4a5e0cce720acfa184,NaN,NaN
1,4d3ab780325ff04df0dc1e45,NaN,NaN
2,583848f313bb7732ce36366c,NaN,NaN
3,57c5bd65498eb3403f51e7e6,NaN,NaN
4,4c5cdff56ebe2d7faa8bd12e,NaN,NaN
...,...,...,...
57,4e5f2cbed4c08cf7f597c801,NaN,NaN
58,4d19e2eb401db60cdd8cfda4,NaN,NaN
59,4cd4f834fb595481b3aedb50,NaN,NaN
60,57aac09e498e56d76d99146e,NaN,NaN


In [111]:

urlvenue = 'https://api.foursquare.com/v2/venues/4d3ab780325ff04df0dc1e45?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_hue, 
    longitude_hue, 
    radius, 
    LIMIT)


Get the venue rating

In [115]:
results_hue2 = requests.get(urlvenue).json()
rating_hue = results_hue2['response']['venue']['rating']
rating_hue

8.1

In [113]:
results_hue2

{'meta': {'code': 200, 'requestId': '602b4c8f051c9e66165360c5'},
 'response': {'venue': {'id': '4d3ab780325ff04df0dc1e45',
   'name': 'Lạc Thiên',
   'contact': {},
   'location': {'address': '6A Đinh Tiên Hoàng',
    'crossStreet': 'Trần Hưng Đạo',
    'lat': 16.468668,
    'lng': 107.585144,
    'labeledLatLngs': [{'label': 'display',
      'lat': 16.468668,
      'lng': 107.585144}],
    'distance': 626,
    'cc': 'VN',
    'city': 'Huế',
    'state': 'Tỉnh Thừa Thiên-Huế',
    'country': 'Việt Nam',
    'formattedAddress': ['6A Đinh Tiên Hoàng (Trần Hưng Đạo)',
     'Huế',
     'Tỉnh Thừa Thiên-Huế',
     'Việt Nam']},
   'canonicalUrl': 'https://foursquare.com/v/l%E1%BA%A1c-thi%C3%AAn/4d3ab780325ff04df0dc1e45',
   'categories': [{'id': '4bf58dd8d48988d14a941735',
     'name': 'Vietnamese Restaurant',
     'pluralName': 'Vietnamese Restaurants',
     'shortName': 'Vietnamese',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vietnamese_',
      'suffix': '.png'}

In [105]:
urlvenue = 'https://api.foursquare.com/v2/venues/4d3ab780325ff04df0dc1e45/tips?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_hue, 
    longitude_hue, 
    radius, 
    LIMIT)

In [107]:
results_hue_tip = requests.get(urlvenue).json()
results_hue_tip['response']['venue']


KeyError: 'venue'

We have our id data which we can iterate thru and get the venue rating and tips

No idea how to do this, might have to add a sleep between each call incase it goes too fast

In [ ]:
# creating map zoom at present Location
map_hue = folium.Map(location=[latitude_hue,longitude_hue],zoom_start=14)

# adding the current location to map
folium.CircleMarker(location=[latitude_hue,longitude_hue],radius=5,color='red',fill=True,fill_color='red').add_to(map_hue)
folium.Marker(location=[latitude_hue,longitude_hue],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map_hue)

# drawing radius of search
folium.Circle(location=[latitude_hue,longitude_hue],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map_hue)

# adding all the popular places around
for lat,lon,name,cate,addr,dist in zip(nearby_hue.lat,nearby_hue.lng,nearby_hue.name,nearby_hue.categories,nearby_hue.address,nearby_hue.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map_hue)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map_hue)

map_hue

# Methodology<a id='methodology' name="methodology"></a>

## Lets plot a bar chart for the Hue data

In [ ]:
count_hue = nearby_hue.groupby('categories').count()
count_hue.sort_values(by = ['name'], inplace=True, ascending=False)
count_hue = count_hue['name']
print("List of placers in Hue")
count_hue

## Lets plot the venues out for Da Nang

In [ ]:
from matplotlib.ticker import StrMethodFormatter
ax = count_hue.plot(kind='barh', figsize=(8, 10), color='#86bf91', zorder=2, width=0.85)

  # Despine
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

  # Switch off ticks
ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

  # Draw vertical axis lines
vals = ax.get_xticks()
for tick in vals:
    ax.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

  # Set x-axis label
ax.set_xlabel("Number of venues", labelpad=20, weight='bold', size=12)

  # Set y-axis label
ax.set_ylabel("Venue type", labelpad=20, weight='bold', size=12)

  # Format y-axis label
ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,g}'))

# Analysis<a id='analysis' name="analysis"></a>

Bar charts go here and counts of venues for Hue and Da Nang

In [ ]:
count_danang = nearby_danang.groupby('categories').count()
count_danang.sort_values(by = ['name'], inplace=True, ascending=False)
count_danang = count_danang['name']
print("List of places in Da Nang")
count_danang

In [ ]:
ax = count_danang.plot(kind='barh', figsize=(8, 10), color='#86bf91', zorder=2, width=0.85)

  # Despine
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

  # Switch off ticks
ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

  # Draw vertical axis lines
vals = ax.get_xticks()
for tick in vals:
    ax.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

  # Set x-axis label
ax.set_xlabel("Number of venues", labelpad=20, weight='bold', size=12)

  # Set y-axis label
ax.set_ylabel("Venue type", labelpad=20, weight='bold', size=12)

  # Format y-axis label
ax.xaxis.set_major_formatter(StrMethodFormatter('{x:,g}'))

In [ ]:
print("There are ",count_danang.shape[0], "places to visit in Da Nang and ",count_hue[0],"places to visit in Hue.")

## Results<a id='results' name="results"></a>

Nothing interesting but the data is confusing a lot different types of resturants for example  fast food, Burger joint and Sandwich place in the Da Nang data.

# Discussion<a id='discussion' name="discussion"></a>

Stuff about US usage verses Vietnam usage, dates of reviews and tips (Argh that is premium API end point call!)

# Conclusion<a id='conclusion' name="conclusion"></a>

Foursquare isn't really used in Vietnam. Maybe Google maps, Tripadvisor, Booking is better for South East  Asia. Plus stuff about dates of ratings and of course Covid19